In [ ]:
import pandas as pd
import configparser
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
default_seed = 1

Because the housing data was ‘.csv’ format, the code to read the csv was used to obtain the database. 

In [ ]:
HouseData = pd.read_csv('/Users/yangruiling/Desktop/iftbigdatacoursework2019/RuilingYang/csv/train.csv')

Then for checking and learning the whole data, I used the head function.

In [ ]:
HouseData.head()

After finding that there are too many columns and I cannot know all about the feature label, the ‘iloc’ code was used to view every labels for each columns. Through using the ‘info’ code, we also can be clear about the data type for each columns.

In [ ]:
HouseData.iloc[:,15:30]

In [ ]:
len(HouseData)

Through using the ‘info’ code, we also can be clear about the data type for each columns.

In [ ]:
HouseData.info()

In order to analysis what features of house can affect the price of the house, the correlation matrix of price and other features was obtained by coding, and sorted the results in descending.

In [ ]:
corr_matrix = HouseData.corr()
corr_matrix["price_doc"].sort_values(ascending=False)

After processing the correlation matrix, we can find that number of rooms and full square are the most positively correlated to price. Therefore, I plot the scatter figure separately for these two features

In [ ]:
plt.scatter(HouseData.num_room,HouseData.price_doc)

According to this results, for number of rooms and price, the relationship was more like classification than linear regression. 

In [ ]:
plt.scatter(HouseData.full_sq,HouseData.price_doc)

For full square and price, the extreme outliers make the relationship very fuzzy.

Then I want to calculate the covariance of number of rooms and price as well as the covariance of full square and price.

In [ ]:
HouseData[['price_doc','num_room' ]].cov()

In [ ]:
HouseData[['price_doc', 'full_sq']].cov()

To measure the relationship deeply, I set the full square and number of rooms as the independent variables: x1 and x2 separately, and I set the price as the dependent variable. 

In [ ]:
x1= 'full_sq'
x2= 'num_room'
y= 'price_doc'

I create two Linear Regression Model to test the possibility that how the independent variable explain the dependent variable.

In [ ]:
parameters = HouseData[[x1, y]]
Linearmodel1 = LinearRegression()
Linearmodel1.fit(parameters[x1].values.reshape(-1, 1), parameters[y])

In [ ]:
alpha1 =Linearmodel1.intercept_
beta1 =Linearmodel1.coef_

In [ ]:
alpha1

In [ ]:
beta1

Because of difference of value range, the alpha and beta were big.

In [ ]:
Linearmodel1.score(parameters[x1].values.reshape(-1, 1), parameters[y])

This score shows that full square cannot explain the price well. Considering about the range between full square and price, I took log for the price and create a new model to estimate.

In [ ]:
Linearmodel1_1 = LinearRegression()
Linearmodel1_1.fit(parameters[x1].values.reshape(-1, 1), np.log(parameters[y]))

In [ ]:
alpha1_1=Linearmodel1_1.intercept_
beta1_1=Linearmodel1_1.coef_

In [ ]:
alpha1_1

In [ ]:
beta1_1

In [ ]:
Linearmodel1_1.score(parameters[x1].values.reshape(-1, 1), np.log(parameters[y]))

This time, the beta and alpha is small, but the score of the model is much lower, which means the problem of low score is not because the range difference between price and full square. According to the scatter figure following, the outliers of price are significant. Therefore I try to remove the extreme value of price and plot the scatter between full square and price.

In [ ]:
plt.scatter(HouseData[x1], np.log(HouseData[y]), marker='o', color='', edgecolors='k')

In [ ]:
HouseData = HouseData[(HouseData[x1] < 300)]
plt.scatter(HouseData[x1], HouseData[y], marker='o', color='', edgecolors='k')

Then I creat a new model without outliers to make the regression.

In [ ]:
HouseDataDrop=HouseData[(HouseData[x1] < 300)]

In [ ]:
parameters1_2 = HouseDataDrop[[x1, y]]
Linearmodel1_2 = LinearRegression()
Linearmodel1_2.fit(parameters1_2[x1].values.reshape(-1, 1), parameters1_2[y])
alpha1_2=Linearmodel1_2.intercept_
beta1_2=Linearmodel1_2.coef_

In [ ]:
print('alpha1_2 = ',alpha1_2, '\n', 'beta1_2 = ',beta1_2)

In [ ]:
Linearmodel1_2.score(parameters1_2[x1].values.reshape(-1, 1), parameters1_2[y])

This score is higher than before, thus the outliers indeed affect the results of regression.

In [ ]:
MeanSSR = np.mean(parameters1_2[y]-model.predict(parameters[x1].values.reshape(-1, 1)))
MeanSSR

Then we start another estimation of the linear regression model from number of rooms and price.

In [ ]:
parameters = HouseData[[x2, y]].dropna()
Linearmodel2 = LinearRegression()
Linearmodel2.fit(parameters[x2].values.reshape(-1, 1), parameters[y])

In [ ]:
alpha3 =Linearmodel2.intercept_
beta3 =Linearmodel2.coef_

In [ ]:
alpha3

In [ ]:
beta3

In [ ]:
Linearmodel2.score(parameters[x2].values.reshape(-1, 1), parameters[y])

In this model, the beta and alpha is still very high because of the range gap. And the score of this mode is 0.2268964974, which is also not good and means it cannot explain the dependent variable.

In [ ]:
plt.scatter(parameters[x2], parameters[y],marker='*', color='r')

Through the results above, we can see that one dimensional regression is not enough to explain the price. Therefore, according to the correlation results above and the life experience, I select twelve feature variables as the independent variables to build the multiple linear regression model with price. 

Because of NA variable, I use ‘loc’ and ‘dropna’ code to process the data to a new database.

In [ ]:
HousePrice= HouseData.loc[:,('num_room','full_sq','area_m','green_zone_part','healthcare_centers_raion','indust_part','children_preschool','shopping_centers_raion','sport_count_5000','sadovoe_km','kremlin_km','zd_vokzaly_avto_km','price_doc')].dropna(how='any')

In [ ]:
HousePrice

In [ ]:
x=HousePrice[['num_room','full_sq','area_m','green_zone_part','healthcare_centers_raion','indust_part','children_preschool','shopping_centers_raion','sport_count_5000','sadovoe_km','kremlin_km','zd_vokzaly_avto_km']]
y=HousePrice['price_doc']

In [ ]:
MultiRegress=LinearRegression()
MultiRegress.fit(x,y)

In [ ]:
Mulalpha=MultiRegress.intercept_
Mulbeta=MultiRegress.coef_

In [ ]:
Mulalpha

In [ ]:
Mulbeta

In [ ]:
MultiRegress.score(x,y)

In this model, the score is 0.503817423, which is much higher than before. However, this result is still not very good. Therefore, through summary the problem above and possible problems, I decide to classify the price into two levels by comparing with the mean of price, and this processing can eliminate the problem of outliers. By classifying the price into two groups, we only consider about how the inputs makes the price of house high or low.

In [ ]:
HouseData.price_doc.describe()

In [ ]:
HouseData['PriceClass']=HouseData.price_doc> np.mean(HouseData.price_doc)

In [ ]:
HouseData['PriceClass']

In [ ]:
HouseData

In [ ]:
HousePrice1= HouseData.loc[:,('num_room','full_sq','area_m','green_zone_part','healthcare_centers_raion','indust_part','children_preschool','shopping_centers_raion','sport_count_5000','sadovoe_km','kremlin_km','zd_vokzaly_avto_km','PriceClass')].dropna(how='any')

In [ ]:
x=HousePrice1[['num_room','full_sq','area_m','green_zone_part','healthcare_centers_raion','indust_part','children_preschool','shopping_centers_raion','sport_count_5000','sadovoe_km','kremlin_km','zd_vokzaly_avto_km']]
y=HousePrice1['PriceClass']

Moreover, considering about the possibility of similar part among the independent variable, I use the principal component analysis (PCA) to remove the correlation among the independent variables.

In [ ]:
noise_var = []
exp_ratio = []
score=[]
for i in range(1, 13):
    PCAmodel = PCA(n_components=i)
    PCAmodel.fit(x, y)
    noise_var.append(PCAmodel.noise_variance_)
    score.append(PCAmodel.score(x,y))


After completing the for loop, I drawn the graphics of estimated noisy variance and the score for PCA model to find the best number of components

In [ ]:
%config InlineBackend.figure_format = 'retina'
fig, ax = plt.subplots()
_ = ax.set_title ("Estimated Noice variance for PCA Model")
_ = ax.plot(range(1, 13), noise_var,color='r')
_ = ax.set_ylabel("estimated noice variance")
_ = ax.set_xlabel('number of components')

In [ ]:
%config InlineBackend.figure_format = 'retina'
fig, ax = plt.subplots()
_ = ax.set_title ("The score for PCA Model")
_ = ax.plot(range(1, 13), score,color='g',marker='o')
_ = ax.set_ylabel("estimated noice variance")
_ = ax.set_xlabel('number of components')

In [ ]:
PCRModel = PCA(n_components= 2)
PCRModel.fit(x, y)
x=PCRModel.transform(x)

In [ ]:
LogicRegress=LogisticRegression()
LogicRegress.fit(x,y)
LogicRegress.score(x,y)

In these two figures, we can obtain that two components are the best choice for this regression model. Then I applied this results to make a new model and calculated the score of this model. The score is 0.575338533, which is higher than all the regression models above.

In the conclusion, although the result is improved, this score is still not very good. We might need to try other regression approaches or other independent variables to explain the price.